### Install the latest .whl package

Check [here](https://pypi.org/project/semantic-link-labs/) to see the latest version.

In [ ]:
%pip install semantic-link-labs

## Requirements

#### App Registration
1. Create an [App Registration](https://learn.microsoft.com/entra/identity-platform/howto-create-service-principal-portal) in the Azure Portal. Note down the 'Application (client) ID' as well as the 'Directory (tenant) ID'.
2. Within 'Manage -> Certificates and secrets', create a new client secret. Copy this down as you will not be able to retrieve it later.
#### Key Vault
1. Create an [Azure Key Vault](https://learn.microsoft.com/azure/key-vault/general/quick-create-portal) within the Azure Portal.
2. Within 'Objects -> Secrets', create 3 secrets (for the ClientID, TenantID, ClientSecret).
#### Groups
1. Create a new group within the Azure Portal. Make sure it is of type 'Security'.
2. Add the app registered (service principal) as a member of this security group.
#### Fabric Admin Portal
1. Add the security group to the setting 'Service principals can use Fabric APIs'.
2. Add the security group to the setting 'Service principals can access read-only admin APIs'.

Here are [additional instructions](https://learn.microsoft.com/fabric/admin/metadata-scanning-enable-read-only-apis) regarding admin read-only APIs. Make sure to read and follow these instructions.

For connecting to Azure Analysis Services (AAS) via Service Principal, add the Service Principal (app) as an admin the AAS instance either in the Azure Portal or using SSMS in this format: `app:<clientid>@<tenantid>`.

**Authentication to retrieve the Key Vault secrets is made using the account which is executing the notebook so please make sure that this account has access to the Key Vault.**

### Import the library and obtain a TokenProvider using Azure Key Vault

In [ ]:
import sempy_labs as labs
from sempy_labs import admin, graph
from sempy_labs.tom import connect_semantic_model

key_vault_uri = '' # Enter your key vault URI
key_vault_tenant_id = '' # Enter the key vault key to the secret storing your Tenant ID
key_vault_client_id = '' # Enter the key vault key to the secret storing your Client ID (Application ID)
key_vault_client_secret = '' # Enter the key vault key to the secret storing your Client Secret

### Use a Service Principal to authenticate to functions

In [ ]:
with labs.service_principal_authentication(
    key_vault_uri=key_vault_uri, 
    key_vault_tenant_id=key_vault_tenant_id,
    key_vault_client_id=key_vault_client_id,
    key_vault_client_secret=key_vault_client_secret):

    df = admin.list_capacities()
    display(df)

In [ ]:
with labs.service_principal_authentication(
    key_vault_uri=key_vault_uri, 
    key_vault_tenant_id=key_vault_tenant_id,
    key_vault_client_id=key_vault_client_id,
    key_vault_client_secret=key_vault_client_secret):

    df = labs.list_subscriptions()
    display(df)

In [ ]:
with labs.service_principal_authentication(
    key_vault_uri=key_vault_uri, 
    key_vault_tenant_id=key_vault_tenant_id,
    key_vault_client_id=key_vault_client_id,
    key_vault_client_secret=key_vault_client_secret):

    labs.suspend_fabric_capacity(
        capacity='',
        azure_subscription_id='',
        resource_group='',
    )

### Using wrapper functions for [Microsoft Graph](https://learn.microsoft.com/graph/overview)

In [ ]:
with labs.service_principal_authentication(
    key_vault_uri=key_vault_uri, 
    key_vault_tenant_id=key_vault_tenant_id,
    key_vault_client_id=key_vault_client_id,
    key_vault_client_secret=key_vault_client_secret):

    df = graph.list_users()
    display(df)

### Use a Service Principal to connect to the Tabular Object Model (also to [Azure Analysis Services](https://learn.microsoft.com/azure/analysis-services/analysis-services-overview))

Note that the Service Principal must have at least read permission to the semantic model to which it is connecting. See [here](https://learn.microsoft.com/power-bi/enterprise/service-premium-service-principal) for more details.

In [ ]:
dataset = '' # Enter the name of the semantic model
workspace = None # Enter the name of the workspace (for Azure Analysis Serivces instance use this format: "asazure://<region>.asazure.windows.net/<server_name>")

In [ ]:
with labs.service_principal_authentication(
    key_vault_uri=key_vault_uri, 
    key_vault_tenant_id=key_vault_tenant_id,
    key_vault_client_id=key_vault_client_id,
    key_vault_client_secret=key_vault_client_secret):

    with connect_semantic_model(dataset=dataset, workspace=workspace, readonly=True) as tom:
        for t in tom.model.Tables:
            print(t.Name)